In [ ]:
import nltk
import string
import re
import os.path

# Load data

The sample text I used still had chapter names and a lot of blank spaces that had to be removed before finetuning the model.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

with open('/content/drive/MyDrive/Colab Notebooks/Sample_Folder/Sample_text.txt', 'r') as file:
  sample = file.read()

# Data Prep / Data Cleaning

In [ ]:
def remove_chapters(text):
    # Find Chapter followed by space and one or more digit
    pattern = re.compile(r'Chapter \d+')

    # Use sub() to replace matched patterns with an empty string
    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text

In [ ]:
text = remove_chapters(sample)

# Remove newlines and carriage returns
text = text.replace('\n', ' ').replace('\r', ' ')

# Remove any leading or trailing white spaces
text = text.strip()

# Convert all text to lowercase
text = text.lower()

# Remove any punctuation
text = text.translate(str.maketrans('', '', string.punctuation))

# Remove any extra whitespace
text = ' '.join(text.split())

# Remove any chapter headings, footnotes, or other non-story text
# This step will depend on the specific structure of your text
# You may need to manually remove these sections or use regular expressions to match and remove them
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Tokenize the text into words
words = nltk.word_tokenize(text)

words_string= ' '.join(words)

print(words_string[:20]) #to check results as you go, use index, don't try and print all words if it's a large document

# Save new file in file path

In [ ]:
save_path = '/content/drive/MyDrive/Colab Notebooks/Sample_Folder/'

name_of_file = "TEXT_transformed"

completeName = os.path.join(save_path, name_of_file+".txt")

toFile = words_string

with open(completeName, "wb") as my_file:
  my_file.write(toFile.encode('utf-8'))

# Get GTP model to finetune later on

I used gtp-2-simple since it's the free version, but if you want a more accurate result, it might be worth comparing it to the paid model.

In [ ]:
!pip install gpt-2-simple

In [ ]:
import gpt_2_simple as gpt2
gpt2.download_gpt2()
import tensorflow as tf

##Check if GPU is enables

In [ ]:
tf.test.gpu_device_name()

##Load pre-trained GPT-2 model

In [ ]:
# Load the pre-trained GPT-2 model
model_name = "124M"
gpt2.download_gpt2(model_name=model_name)
sess = gpt2.start_tf_sess()

file_name = 'TEXT_transformed.txt'

#Finetune model

- Here we specify the name of the run. I like including the date
- With save_every we can save the model after a specific number of sets
- By default, the gpt2.finetune function saves the 5 most recent checkpoints and deletes older checkpoints to save disk space. You can change this behavior by setting the keep_checkpoint_max parameter to a different value.
The advantage with having a name of the run, means we can reload the trained model and finetune it further once we get more data.

In [ ]:
name_run = '2xxxxx_sample_vxx'

In [ ]:
gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name= name_run,
              print_every=10,
              sample_every=200,
              save_every=50
              )

#Generate text
- temperature: Temperature is a hyperparameter that controls the randomness of the generated text. A higher temperature (e.g., 0.7) makes the output more random and creative, while a lower temperature (e.g., 0.2) makes the output more focused and deterministic.
- return_as_list: if set to true it returns the text as a list of strings.

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=name_run) #load finetuned model
gentext1 = gpt2.generate(sess, run_name=name_run) #generate from the trained model
gentext2 = gpt2.generate(sess, length=1023, temperature= 0.7,  return_as_list=True, run_name= name_run) #generate from the trained model